Used to only scrape data from profiles

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import requests
import urllib.request
import urllib.parse
import urllib.error
import ssl
import json
import pandas as pd
import numpy as np
from datetime import datetime
import time
import tqdm
import traceback
import requests
import random
import pickle
import urllib.request as req

In [2]:
data = []

In [3]:
df = pd.read_excel("Cannabis Retailer Instagram Study_IG links only_3.5.2020.xlsx")

In [4]:
profiles = df['profile'].to_list()
for i in tqdm.trange(len(profiles)):
    link = profiles[i]
    if '?' in link:
        profiles[i] = link[:link.find('?')]
    if profiles[i][-1] != '/':
        profiles[i] += '/'

100%|████████████████████████████████████████████████████████████████████████████████████████| 183/183 [00:00<?, ?it/s]


In [6]:
class InstaScraper:
    firstPage = None
    
    def __init__(self):
        self.firstPage = None
        self.ctx = ssl.create_default_context()
        self.ctx.check_hostname = False
        self.ctx.verify_mode = ssl.CERT_NONE
    
    def getUserMetadata(self, url, userInfo):
        html = urllib.request.urlopen(url, context=self.ctx).read()
        pageData = json.loads(html)
        pageData = pageData['graphql']['user']
        # store a temp copy for later
        self.firstPage = pageData
        
        userInfo['name'] = pageData['full_name']
        userInfo['isPrivate'] = pageData['is_private']
        if userInfo['isPrivate']:
            return;
        userInfo['hasNextPage'] = pageData['edge_owner_to_timeline_media']['page_info']['has_next_page']
        userInfo['bio'] = pageData['biography']
        userInfo['website'] = pageData['external_url']
        userInfo['followerCount'] = pageData['edge_followed_by']['count']
        userInfo['followingCount'] = pageData['edge_follow']['count']
        userInfo['id'] = pageData['id']
        userInfo['isBusinessAccount'] = pageData['is_business_account']
        userInfo['businessCategory'] = pageData['business_category_name']
        userInfo['isVerified'] = pageData['is_verified']
        userInfo['profilePicURL'] = pageData['profile_pic_url_hd']
        userInfo['userName'] = pageData['username']
        userInfo['FacebookPage'] = pageData['connected_fb_page']
        userInfo['postCount'] = pageData['edge_owner_to_timeline_media']['count']

    def getlinks(self, url, count, userInfo, endCursor = None):

        if count == 0 and self.firstPage:
            pageData = self.firstPage
            self.firstPage = None
        else:
            if endCursor:
                temp_url = 'https://instagram.com/graphql/query/?query_id=17888483320059182&id=' + userInfo['id'] + '&first=50&after=' + endCursor
                html = urllib.request.urlopen(temp_url, context=self.ctx).read()
            else:
                html = urllib.request.urlopen(url, context=self.ctx).read()
            pageData = json.loads(html)
            if 'graphql' in pageData:
                pageData = pageData['graphql']['user']
            else:
                pageData = pageData['data']['user']
        
        endCursor = pageData['edge_owner_to_timeline_media']['page_info']['end_cursor']
        userInfo['hasNextPage'] = pageData['edge_owner_to_timeline_media']['page_info']['has_next_page']
            
        if 'posts' not in userInfo:
            userInfo['posts'] = []
        for node in pageData['edge_owner_to_timeline_media']['edges']:
            post = node['node']
            temp = {}
            try:
                temp['linkCode'] = post['shortcode']
                temp['isVideo'] = post['is_video']
                temp['postType'] = post['__typename']
                temp['postId'] = post['id']
                
                if post['edge_media_to_caption']['edges']:
                    temp['postCaption'] = post['edge_media_to_caption']['edges'][0]['node']['text']
                else:
                    temp['postCaption'] = ''
                    
                temp['imageLink'] = post['display_url']                
                temp['postLink'] = "https://www.instagram.com/p/" + temp['linkCode']
                temp['commentCount'] = post['edge_media_to_comment']['count']
                temp['timeStamp'] = post['taken_at_timestamp']
                temp['numLikes'] = post['edge_media_preview_like']['count']
                
                if temp['postType'] == 'GraphSidecar' or temp['isVideo']:
                    html_temp =  urllib.request.urlopen(temp['postLink'] + '/?__a=1', context=self.ctx).read()
                    pageData_temp = json.loads(html_temp)
                    pageData_temp = pageData_temp['graphql']['shortcode_media']
                    
                    if temp['isVideo']:
                        temp['videoURL'] = pageData_temp['video_url']
                        temp['videoViewCount'] = pageData_temp['video_view_count']
                    else:
                        edgeLinks = []
                        for node in pageData_temp['edge_sidecar_to_children']['edges']:
                            edgeLinks.append(node['node']['display_url'])
                        temp['imgURLList'] = edgeLinks
                userInfo['posts'].append(temp)       
            except Exception as e:
                print(e)
                print("link: " + temp['linkCode'])
            count+=1

        return endCursor, count 

    def main(self, profiles, returnData):
        i = 0
        for profile in profiles:
            count = 0
            userInfo = {}
            extractLink = profile + '?__a=1'
            endCursor = ''
            userInfo['dfIndex'] = i
            userInfo['dfMasterNo'] = df.loc[i][1]
            i+=1
            
            try:
                self.getUserMetadata(extractLink, userInfo)
            except Exception as e:
                print("%d. Error in getting user metadata: " %(i) + str(e))
                print(profile)
                traceback.print_exc()
                userInfo['deleted'] = True
                returnData.append(userInfo)
                continue
                
            print(str(i) + ". Current profile: " + userInfo['name'])
            if(userInfo['isPrivate']):
                print("User is private, skipping user")
                returnData.append(userInfo)
                continue
            
            print("Number of posts: " + str(userInfo['postCount']))
            while count < userInfo['postCount']:
                try:
                    temp_endCursor, temp_count = self.getlinks(extractLink, count, userInfo, endCursor)
                    if count == temp_count:
                        print("ERR: Value unchanged")
                    count = temp_count
                    endCursor = temp_endCursor
                    print(count)
                except Exception as e:
                    print("Error: " + str(e))
                    time.sleep(random.uniform(5, 15))
                if not userInfo['hasNextPage']:
                    break
            returnData.append(userInfo)
            sleepTime = random.uniform(3, 6)
            print('sleeping %f seconds' %(sleepTime))
            time.sleep(sleepTime)

In [7]:
obj = InstaScraper()

In [9]:
data = []
obj.main(profiles, data)

1. Current profile: dosist
Number of posts: 172
12
62
112
162
172
sleeping 3.943198 seconds
2. Current profile: Buds & Roses™️
Number of posts: 5
5
sleeping 4.189105 seconds
3. Current profile: MMD MARINA DEL REY
Number of posts: 65
12
62
65
sleeping 3.937766 seconds
4. Current profile: ChronicPractor Caregiver
Number of posts: 109
12
62
109
sleeping 3.488399 seconds
5. Current profile: 🌱Coast To Coast Canoga 🌱
Number of posts: 192
12
62
112
162
191
sleeping 5.767490 seconds
6. Current profile: CompassionatePatientResources
Number of posts: 70
12
62
70
sleeping 4.155908 seconds
7. Current profile: Green Cross Of Torrance
Number of posts: 15
12
15
sleeping 5.695056 seconds
8. Current profile: Daddys Pipes Smoke Shop Est.98
Number of posts: 2003
12
62
112
162
212
262
312
362
412
462
512
562
612
662
712
762
812
862
912
962
1012
1062
1112
1162
1212
1262
1312
1362
1412
1462
1512
1562
1612
1662
1712
1762
1812
1862
1912
1962
2003
sleeping 3.964067 seconds
9. Current profile: Delta 9 THC
Numbe

Traceback (most recent call last):
  File "<ipython-input-6-85b140b82376>", line 110, in main
    self.getUserMetadata(extractLink, userInfo)
  File "<ipython-input-6-85b140b82376>", line 11, in getUserMetadata
    html = urllib.request.urlopen(url, context=self.ctx).read()
  File "D:\Anaconda\lib\urllib\request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "D:\Anaconda\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "D:\Anaconda\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "D:\Anaconda\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "D:\Anaconda\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "D:\Anaconda\lib\urllib\request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 404: Not Found


12. Error in getting user metadata: HTTP Error 404: Not Found
https://www.instagram.com/theplug_hollywood/


Traceback (most recent call last):
  File "<ipython-input-6-85b140b82376>", line 110, in main
    self.getUserMetadata(extractLink, userInfo)
  File "<ipython-input-6-85b140b82376>", line 11, in getUserMetadata
    html = urllib.request.urlopen(url, context=self.ctx).read()
  File "D:\Anaconda\lib\urllib\request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "D:\Anaconda\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "D:\Anaconda\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "D:\Anaconda\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "D:\Anaconda\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "D:\Anaconda\lib\urllib\request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 404: Not Found


13. Current profile: GREENWOLF LA
Number of posts: 23
12
23
sleeping 4.034889 seconds
14. Current profile: Medical Marijuana💫🚀🌕
Number of posts: 61
12
61
sleeping 3.415189 seconds
15. Error in getting user metadata: Expecting value: line 1 column 1 (char 0)
https://www.instagram.com/kush.alley/


...

152. Current profile: KUSHLAND
Number of posts: 196
12
62
112
162
196
sleeping 4.082509 seconds
153. Error in getting user metadata: HTTP Error 404: Not Found
https://www.instagram.com/budhub4200/


Traceback (most recent call last):
  File "<ipython-input-6-85b140b82376>", line 110, in main
    self.getUserMetadata(extractLink, userInfo)
  File "<ipython-input-6-85b140b82376>", line 11, in getUserMetadata
    html = urllib.request.urlopen(url, context=self.ctx).read()
  File "D:\Anaconda\lib\urllib\request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "D:\Anaconda\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "D:\Anaconda\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "D:\Anaconda\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "D:\Anaconda\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "D:\Anaconda\lib\urllib\request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 404: Not Found


154. Current profile: Mr.T Collective PRE-ICO
Number of posts: 44
12
44
sleeping 4.016628 seconds
155. Current profile: OGXPRESS
Number of posts: 26
12
26
sleeping 3.827735 seconds
156. Current profile: TOPSHELF
Number of posts: 112
12
62
112
sleeping 4.298372 seconds
157. Current profile: Olympic 25
Number of posts: 57
12
57
sleeping 5.794219 seconds
158. Current profile: 
Number of posts: 36
12
36
sleeping 4.141931 seconds
159. Current profile: Perennial Holistic
Number of posts: 256
12
62
112
162
212
256
sleeping 3.098936 seconds
160. Current profile: Pot Spot Collective
Number of posts: 8
8
sleeping 4.050404 seconds
161. Current profile: Royal Greens LA
Number of posts: 57
12
57
sleeping 5.141931 seconds
162. Current profile: RxEclusive
Number of posts: 5
5
sleeping 3.080937 seconds
163. Current profile: Santee.greens
Number of posts: 19
12
19
sleeping 4.452959 seconds
164. Current profile: Scooters&HootersLA
User is private, skipping user
165. Current profile: SHIVAS DTLA
Number o

Traceback (most recent call last):
  File "<ipython-input-6-85b140b82376>", line 110, in main
    self.getUserMetadata(extractLink, userInfo)
  File "<ipython-input-6-85b140b82376>", line 11, in getUserMetadata
    html = urllib.request.urlopen(url, context=self.ctx).read()
  File "D:\Anaconda\lib\urllib\request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "D:\Anaconda\lib\urllib\request.py", line 531, in open
    response = meth(req, response)
  File "D:\Anaconda\lib\urllib\request.py", line 641, in http_response
    'http', request, response, code, msg, hdrs)
  File "D:\Anaconda\lib\urllib\request.py", line 569, in error
    return self._call_chain(*args)
  File "D:\Anaconda\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "D:\Anaconda\lib\urllib\request.py", line 649, in http_error_default
    raise HTTPError(req.full_url, code, msg, hdrs, fp)
urllib.error.HTTPError: HTTP Error 404: Not Found


171. Current profile: 🎓 University 20 Cap 🦄🌿
Number of posts: 245
12
62
112
162
212
244
sleeping 3.088912 seconds
172. Current profile: West Coast Collective
Number of posts: 19
12
19
sleeping 3.427445 seconds
173. Current profile: Western Top shop
User is private, skipping user
174. Current profile: Breaking Bud 20 Cap
Number of posts: 2
2
sleeping 5.484493 seconds
175. Current profile: 🌱Coast To Coast Los Angeles🌱
Number of posts: 132
12
62
112
132
sleeping 3.130112 seconds
176. Current profile: church herbs
User is private, skipping user
177. Current profile: Green Bear
User is private, skipping user
178. Current profile: Gbccollective
User is private, skipping user
179. Current profile: House of OG's
Number of posts: 8
8
sleeping 5.865238 seconds
180. Current profile: ⚜️Infinite Wellness⚜️
Number of posts: 27
12
27
sleeping 5.009802 seconds
181. Current profile: Mary J’s
Number of posts: 62
12
62
sleeping 5.370015 seconds
182. Current profile: Save Greens
Number of posts: 26
12
26


In [12]:
file = open('instagram_profiles.pickle', 'wb')
pickle.dump(data, file)
file.close()

In [5]:
file = open('instagram_profiles.pickle', 'rb')
data = pickle.load(file)
file.close()

In [6]:
count_deleted = 0
count_private = 0
count_business = 0
count_verified = 0
count_fb = 0
count_postmismatch = 0
for user in data:
    if user.get('isPrivate'):
        count_private += 1
    if user.get('deleted'):
        count_deleted += 1
    if user.get('isBusinessAccount'):
        count_business += 1
    if user.get('isVerified'):
        count_verified+=1
    if user.get('FacebookPage'):
        count_fb += 1
    if 'posts' in user and len(user['posts']) != user['postCount']:
        count_postmismatch += 1

In [7]:
count_deleted, count_private, count_business, count_verified, count_fb, count_postmismatch

(23, 9, 95, 4, 0, 12)

In [8]:
cleanData = []
cols = []
for user in data:
    if user.get('isPrivate') or user.get('deleted'):
        continue
    else:
        cleanData.append(user)
        cols.extend(list(user.keys()))
        cols = np.unique(cols).tolist()

In [19]:
file = open('clean_instagram_profiles.pickle', 'wb')
pickle.dump(cleanData, file)
file.close()

In [14]:
df = pd.DataFrame(cleanData, columns=cols)

In [15]:
df.head()

,FacebookPage,bio,businessCategory,dfIndex,dfMasterNo,followerCount,followingCount,hasNextPage,id,isBusinessAccount,isPrivate,isVerified,name,postCount,posts,profilePicURL,userName,website
0,None,A health & wellness company providing plant-ba...,Personal Goods & General Merchandise Stores,0,2,75306,598,False,2093873363,True,False,True,dosist,172,"[{'linkCode': 'B9kAcy0lew9', 'isVideo': True, ...",https://scontent-lax3-2.cdninstagram.com/v/t51...,dosist,http://dosist.com/find-us
1,None,The official IG of Buds & Roses as of 11/24/19...,None,1,4,964,35,False,9515730312,False,False,False,Buds & Roses™️,5,"[{'linkCode': 'B9XnAwkB4_X', 'isVideo': True, ...",https://scontent-lax3-2.cdninstagram.com/v/t51...,budsandroses420,None
2,None,MMD MARINA DEL REY is the 4th @mmdshops to ope...,None,2,9,978,793,False,11844055247,False,False,False,MMD MARINA DEL REY,65,"[{'linkCode': 'B9gVPUOBQZh', 'isVideo': False,...",https://scontent-lax3-2.cdninstagram.com/v/t51...,mmdmarinadelrey,http://www.mmdshops.com/
3,None,Finest strands of Indicas and Sativas\n5751 W....,None,3,12,108,169,False,454105637,False,False,False,ChronicPractor Caregiver,109,"[{'linkCode': 'lbNoHGCfud', 'isVideo': False, ...",https://scontent-lax3-2.cdninstagram.com/v/t51...,chronicpractor,None
4,None,Home of California’s Best Cannabis\nA.K.A “Coa...,Personal Goods & General Merchandise Stores,4,13,2950,426,False,7492039902,True,False,False,🌱Coast To Coast Canoga 🌱,192,"[{'linkCode': 'B9mkf2zBKl3', 'isVideo': False,...",https://scontent-lax3-2.cdninstagram.com/v/t51...,3c.coast,None


In [16]:
df = df.drop(['FacebookPage', 'isPrivate', 'hasNextPage'], axis=1)
# df = df.drop(['FacebookPage', 'dfIndex', 'isPrivate', 'hasNextPage', 'posts'], axis=1)
# df = df.rename(index=str, columns={'dfMasterNo':'master_list_retail_number'})

In [17]:
df.head()

,bio,businessCategory,dfIndex,dfMasterNo,followerCount,followingCount,id,isBusinessAccount,isVerified,name,postCount,posts,profilePicURL,userName,website
0,A health & wellness company providing plant-ba...,Personal Goods & General Merchandise Stores,0,2,75306,598,2093873363,True,True,dosist,172,"[{'linkCode': 'B9kAcy0lew9', 'isVideo': True, ...",https://scontent-lax3-2.cdninstagram.com/v/t51...,dosist,http://dosist.com/find-us
1,The official IG of Buds & Roses as of 11/24/19...,None,1,4,964,35,9515730312,False,False,Buds & Roses™️,5,"[{'linkCode': 'B9XnAwkB4_X', 'isVideo': True, ...",https://scontent-lax3-2.cdninstagram.com/v/t51...,budsandroses420,None
2,MMD MARINA DEL REY is the 4th @mmdshops to ope...,None,2,9,978,793,11844055247,False,False,MMD MARINA DEL REY,65,"[{'linkCode': 'B9gVPUOBQZh', 'isVideo': False,...",https://scontent-lax3-2.cdninstagram.com/v/t51...,mmdmarinadelrey,http://www.mmdshops.com/
3,Finest strands of Indicas and Sativas\n5751 W....,None,3,12,108,169,454105637,False,False,ChronicPractor Caregiver,109,"[{'linkCode': 'lbNoHGCfud', 'isVideo': False, ...",https://scontent-lax3-2.cdninstagram.com/v/t51...,chronicpractor,None
4,Home of California’s Best Cannabis\nA.K.A “Coa...,Personal Goods & General Merchandise Stores,4,13,2950,426,7492039902,True,False,🌱Coast To Coast Canoga 🌱,192,"[{'linkCode': 'B9mkf2zBKl3', 'isVideo': False,...",https://scontent-lax3-2.cdninstagram.com/v/t51...,3c.coast,None


In [18]:
df.to_csv('users.csv', index=False)

In [53]:
# df.to_excel('userDetails.xlsx', index=False)